# Image Processing SS 20 - Assignment - 05

### Deadline is 27.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import chelsea
from skimage.color import rgb2gray

import random
from io import BytesIO
    
from PIL import Image
import itertools

pylab.rcParams['figure.figsize'] = (12, 8)   # This makes the plot bigger

# Exercise 1 - Filters - 5 Points

Implement and apply the filters below to noisy images. Plot the noisy image vs the filtered one

In [ ]:
img = rgb2gray(chelsea() / 255.)
img.shape

In [ ]:
def salt_peper(img, p):
    peper_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    salt_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    img_noise = img.copy()
    img_noise[peper_mask] = 0
    img_noise[salt_mask] = 1
    return img_noise

salt_peper_noises = [0.01, 0.03, 0.1]
imgs_salt_peper = [salt_peper(img, p) for p in salt_peper_noises]
for img_salt_peper in imgs_salt_peper:
    plt.imshow(img_salt_peper, cmap='gray')
    plt.show()

In [ ]:
def gaussian_noise(img, std):
    return np.clip(img + np.random.normal(0, std, img.shape), 0, 1)

gaussian_noises = (0.05, 0.10, 0.2)
imgs_gaussian_noise = [gaussian_noise(img, s) for s in gaussian_noises]
for img_gaussian in imgs_gaussian_noise:
    plt.imshow(img_gaussian, cmap='gray')
    plt.show()

In [ ]:
# implement the filters:

#create 3x3 matrix
def arithmetic_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
        for j in range(1, y - 1):
            val = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    val += img[i + k, j + l]
            out[i, j] = val / (1/9)
    return out

#create 3x3 matrix
def geometric_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
        for j in range(1, y - 1):
            val = 1
            for k in range(-1, 2):
                for l in range(-1, 2):
                    val *= img[i + k, j + l]
            out[i, j] = val ** (1/9)
    return out

#create 3x3
def harmonic_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
       for j in range(1, y - 1):
           val = 0
           for k in range(-1, 2):
               for l in range(-1, 2):
                   val += ( 1 / (img[i + k, j + l]) )
           out[i, j] = 9 / val
    return out

def contraharmoic_mean(img, q=-1):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(x - 1):
        for j in range(y - 1):
            counter = 0
            caller = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    counter += (img[i + k, j + l] ** (q + 1))
                    caller += (img[i + k, j + l] ** (q))
            out[i, j] = counter / caller
    return out

def adaptive_mean(img):
    return img

def adaptive_median(img):
    return img


In [ ]:
filters = [arithmetic_mean, geometric_mean, contraharmoic_mean, 
           adaptive_mean, adaptive_median]
for filter in filters:
    for sp, img_sp in zip(salt_peper_noises, imgs_salt_peper):
        plt.suptitle(filter.__name__ + ", salt peper noise: {}".format(sp))
        plt.subplot(221)
        plt.imshow(img_sp, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_sp.copy()), cmap='gray')
        plt.show()


In [ ]:
for filter in filters:
    for gn, img_gn in zip(gaussian_noises, imgs_gaussian_noise):
        plt.suptitle(filter.__name__ + ", gaussian noise: {}".format(gn))
        plt.subplot(221)
        plt.imshow(img_gn, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_gn.copy()), cmap='gray')
        plt.show()

# Exercise 2 - Sharpening - 5 Points

Sharpen the image `img_blurry`. You need to implement the `sharp_laplacian` and `unsharp_masking` functions. You might
consider some processing steps before sharpening.

In [ ]:
from skimage.filters import gaussian

In [ ]:
img_blurry = gaussian_noise(gaussian(img , 1.6), 0.07)
plt.imshow(img_blurry, cmap='gray')

In [ ]:
# ATTENTION: First call one of the filter func to remove pepper 
def sharp_laplacian(img_unfiltered, w):
    """Perform sharpening with a laplacian filter"""

    img = harmonic_mean(img_unfiltered)
    x, y = img.shape
    laplacian_filter = np.zeros_like(img)
    result = np.zeros_like(img)

    for i in range(1, x - 1):
        for j in range(1, y - 1):
            pixel_calc = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    pixel_calc += w[k, l] * img[i - k, j - l]
                laplacian_filter[i, j] = pixel_calc   
    for i in range(1, x - 1):
        for j in range(1, y - 1):
            result[i,j] = img[i,j] + laplacian_filter[i,j]

    return result

#Crop handling, 3x3 Box Blur
# ATTENTION: First call one of the filter func to remove gaussian noise, here harmonic_mean but in my opinion the result is more blured
def unsharp_masking(img_unfiltered, w):
    """Perform sharpening by unsharp masking"""

    img = harmonic_mean(img_unfiltered)
    plt.imshow(img, cmap='gray')
    plt.show()

    #img = img_unfiltered.copy()

    #1nd step: blur the original
    blurry_img = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x-1): #exclude edge
        for j in range(1, y-1): #exclude edge
            pixel_calc = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    pixel_calc += w[k, l] * img[i - k, j - l]
            blurry_img[i, j] = pixel_calc

    plt.imshow(blurry_img, cmap='gray')
    plt.show()
    

    #2nd step: subtract the blurred image from the original
    sub_difference = np.zeros_like(img)
    for i in range(1, x-1):
        for j in range(1, y-1):
            sub_difference[i, j] = img[i, j] - blurry_img[i, j]
    
    plt.imshow(sub_difference, cmap='gray')
    plt.show()
    
    #3nd step: add the result to the original
    result = np.zeros_like(img)
    for i in range(1, x-1):
        for j in range(1, y-1):
            result[i, j] = img[i, j] + sub_difference[i, j] #if - instead + result seems better
    
    return result

In [ ]:
w = np.array([ [0, 1, 0], [1, -4, 1], [0, 1, 0] ])
plt.imshow(sharp_laplacian(img_blurry, w), cmap='gray')
plt.show()

In [ ]:
w = (1 / 9) * np.array([ [1,1,1], [1,1,1], [1,1,1] ]) #blurry matrix
#w = (1 / 16) * np.array([ [1,2,1], [2,4,2], [1,2,1] ]) #blurry matrix
plt.imshow(unsharp_masking(img_blurry, w), cmap='gray')
plt.show()

#Why is the result of the example != from the slides, although the example of the slide is correct?
# test = np.array([ [7, 0, 1, 1], [3, 5, 7, 4], [4, 6, 3, 0], [6, 2, 9, 0] ])
# print(unsharp_masking(test, w))

In [ ]:
#--------------Testing harmonic_mean------------------------

#create 3x3, normaly in the result matrix should be floats, but there are ints
def t_harmonic_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
       for j in range(1, y - 1):
           val = 0
           for k in range(-1, 2):
               for l in range(-1, 2):
                   val += ( 1 / (img[i + k, j + l]) )
           out[i, j] = 9 / val
    return out

arr1 = np.array([[1, 3, 27],  
        [3, 4, 6],  
        [7, 6, 3],  
        [3, 6, 8]])  

t_harmonic_mean(arr1)